In [ ]:
import pandas as pd
import numpy as np
from glob import glob
import os
import pandas as pd
import numpy as np
import plotnine as p9
import matplotlib.pyplot as plt
import ast

In [ ]:
date_time = "2022-12-07_12:41"


coef_df = pd.read_csv(f"/home/da2343/cs685_fall22/my_pearson_multicol/lasso_coef/lasso_coef_{date_time}.csv")

# col_names_list = coef_df.columns.str.replace('f__', '').to_list()
# coef_df.columns = [i for i in range(len(col_names_list))]

# remove some columns that are not needed
coef_df_filtered = coef_df.drop(columns=["algorithm", "data_set_name"])
fold_id_df_filtered = coef_df_filtered[coef_df_filtered["fold_id"] == 2]
fold_id_df_filtered = fold_id_df_filtered.sort_values(by="index_of_pred_col")


fold_id_df_filtered = fold_id_df_filtered.drop(columns=["fold_id", "index_of_pred_col"])
col_names_list = fold_id_df_filtered.columns.str.replace('f__', '').to_list()

fold_id_df_filtered.columns = [i for i in range(len(col_names_list))]
fold_id_df_filtered = fold_id_df_filtered.reset_index(drop=True)
print(fold_id_df_filtered)

df_upper = fold_id_df_filtered
df_upper = df_upper.where(np.triu(np.ones(df_upper.shape)).astype(bool))
df_upper = df_upper.stack().reset_index()
df_upper.columns = ['var1','var2','Value']
print(df_upper)


df_lower = fold_id_df_filtered
df_lower = df_lower.where(np.tril(np.ones(df_lower.shape)).astype(bool))
df_lower = df_lower.stack().reset_index()
df_lower.columns = ['var1','var2','Value']
print(df_lower)

In [ ]:
def draw_coefficient_matrix(local_df, title, col_names):
    import numpy as np
    import seaborn as sns
    import matplotlib.pyplot as plt

    local_df.columns = col_names
    local_df.index = col_names
    sns.set_theme(style="white")

    # Generate a mask for the upper triangle
    mask = np.triu(np.ones_like(df, dtype=bool))

    # Generate a mask for the whole matrix
    mask = np.zeros_like(local_df, dtype=bool)

    # Set up the matplotlib figure
    f, ax = plt.subplots(figsize=(11, 9))

    #set title
    plt.title(title, fontsize=20)

    # change the angle of the x axis labels
    plt.xticks(rotation=-40)


    # Generate a custom diverging colormap
    cmap = sns.diverging_palette(230, 20, as_cmap=True)

    # Draw the heatmap with the mask and correct aspect ratio
    sns.heatmap(local_df, mask=mask, cmap=cmap, vmin=-1.0, vmax=1.0, center=0,
                square=True, linewidths=.5, cbar_kws={"shrink": .5})

def draw_network_graph(links_filtered, title, col_names_list):
    import matplotlib.pyplot as plt
    import igraph as ig
    
    # number of unique variables in the links_filtered var1 and var2 columns
    n_vertices = len(links_filtered)   
    edges = [(int (links_filtered['var1'][i]) , int(links_filtered['var2'][i])) for i in range(len(links_filtered))]
    # filter edges list to remove same edges (i.e. (1,2) and (2,1) are the same edge)
    edges_filtered = []
    for edge in edges:
        (v1, v2) = edge
        if (v2, v1) not in edges_filtered:
            edges_filtered.append(edge)
    # edges_filtered = []         
    graph = ig.Graph(n_vertices, edges_filtered)
    graph["title"] = title
    graph.vs["name"] = col_names_list
    # give each vertex a unique color

    #generate 20 different colors using color codes
    graph.vs["color"] = ['lightblue', 'orange', 'lightgreen', 'pink', 'gray']
    # graph.vs["color"] = colors

    # print(colors)

    #TODO: Check if this is the right way to do this
    graph.es["sign"] = [ True if links_filtered['value'][i] > 0 else False for i in range(len(links_filtered)) ]
    fig, ax = plt.subplots(figsize=(10,10))
    ax.set_title(graph["title"], fontsize=25)

    ig.plot(
        graph,
        target=ax,
        # layout="circle", # print nodes in a circular layout
        vertex_size=0.3,
        # vertex_color = graph.vs["color"],
        vertex_frame_width=0.2,
        vertex_frame_color="white",
        vertex_label=graph.vs["name"],
        vertex_label_size=19.0,
        edge_width=[2 if sign else 1 for sign in graph.es["sign"]],
        edge_color=["#C25A40" if sign else "#AAA" for sign in graph.es["sign"]],
        vertex_label_color = "white",
    )
    plt.show()


date_time = "2022-12-07_12:41"

coef_df = pd.read_csv(f"/home/da2343/cs685_fall22/my_pearson_multicol/lasso_coef/lasso_coef_{date_time}.csv")

# col_names_list = coef_df.columns.str.replace('f__', '').to_list()
# coef_df.columns = [i for i in range(len(col_names_list))]

# remove some columns that are not needed
coef_df_filtered = coef_df.drop(columns=["algorithm", "data_set_name"])
fold_id_list = coef_df["fold_id"].unique()

for fold_id in fold_id_list:
    fold_id_df_filtered = coef_df_filtered[coef_df_filtered["fold_id"] == fold_id]
    # arrange the fold_id_df_filtered according to ascending order of `index_of_pred_col` column
    fold_id_df_filtered = fold_id_df_filtered.sort_values(by="index_of_pred_col")
    fold_id_df_filtered = fold_id_df_filtered.drop(columns=["fold_id", "index_of_pred_col"])
    
    col_names_list = fold_id_df_filtered.columns.str.replace('f__', '').to_list()

    fold_id_df_filtered.columns = [i for i in range(len(col_names_list))]
    fold_id_df_filtered = fold_id_df_filtered.reset_index(drop=True)
    # print(fold_id_df_filtered)
    coef_df = fold_id_df_filtered
    # draw_coefficient_matrix(fold_id_df_filtered, f"Lasso CV Crohns Dataset Fold {fold_id} Coefficient Matrix", col_names_list)
    
    df_upper = fold_id_df_filtered
    df_upper = df_upper.where(np.triu(np.ones(df_upper.shape)).astype(bool))
    df_upper = df_upper.stack().reset_index()
    df_upper.columns = ['var1','var2','Value']
    
    df_lower = fold_id_df_filtered
    df_lower = df_lower.where(np.tril(np.ones(df_lower.shape)).astype(bool))
    df_lower = df_lower.stack().reset_index()
    df_lower.columns = ['var2','var1','Value']

    # create a new dataframe with the mean of Value for each pair of var1 and var2
    df = pd.concat([df_upper, df_lower], ignore_index=True)
    df = df.groupby(['var1', 'var2'] ).mean().reset_index()
    
    #save the df to a csv file
    df.to_csv(f"/home/da2343/cs685_fall22/my_pearson_multicol/lasso_coef/crohns_coef_{date_time}_fold_{fold_id}.csv", index=False)
    
    print(df)
    
    # draw_network_graph(df, f"Lasso CV Crohns Dataset Fold {fold_id} Network Graph", col_names_list)
    

    #combine df_upper and df_lower
    # df_combined = pd.concat([df_upper, df_lower], ignore_index=True)
    # find the mean of the values of each pair of var1 and var2
    # df_combined = df_combined.groupby(["var1", "var2"]).mean().reset_index()
    

# print(df)
    